In [53]:
import pandas as pd
import datetime
import math

# Managing Linear Risk

## Calibrating the Exchange Rate Process

##### a) For θ, you need to refer to the interest rate parity and estimate θ using the forward quotes from Table 1. Note that this a “forward-looking” approach.

In [7]:
forward_quotes = pd.read_csv("FE535_Forward_Prices.csv")
spot_rate = 1.2273

In [49]:
def calculate_theta(bid, ask, T):
    forward_rate = (bid + ask) / 2  # Using the midpoint of bid and ask as forward rate
    return math.log(forward_rate / spot_rate) / T
thetas = []
for contract,bid,ask in zip(forward_quotes["Name"],forward_quotes["Bid"],forward_quotes["Ask"]):
    if contract.split()[1] == '1Y FWD':
        T = 1
    else:
        T = int(contract.split()[1][:-1]) / 12  # Convert maturity to years
    theta = calculate_theta(bid, ask, T)
    thetas.append(theta)
forward_quotes["Theta"] = thetas
forward_quotes

,Name,Bid,Ask,High,Low,Theta
0,GBPUSD 1M FWD,1.90,2.20,2.08,1.95,6.156278
1,GBPUSD 2M FWD,5.48,6.28,5.87,4.82,9.400441
2,GBPUSD 3M FWD,8.19,8.49,8.32,7.63,7.664986
3,GBPUSD 4M FWD,10.61,11.11,10.92,9.93,6.540809
4,GBPUSD 5M FWD,11.68,16.68,14.27,12.86,5.872838
5,GBPUSD 6M FWD,15.17,18.17,16.94,16.00,5.217588
6,GBPUSD 7M FWD,17.40,21.80,20.24,18.81,4.749794
7,GBPUSD 8M FWD,19.66,24.61,23.20,21.59,4.338515
8,GBPUSD 9M FWD,23.71,24.87,26.17,23.60,3.980331
9,GBPUSD 10M FWD,26.07,27.45,29.03,26.47,3.698510


##### b) For foreign exchange rates, it is common to relate to the interbank lending rate in terms risk-free rate. In particular the data file named FE535_Libor_USD_GBP.csv contains LIBOR rates for both USD and GBP. How does your calibrated θ compare with LIBOR rates? (10 Points)

In [45]:
libor_rates = pd.read_csv("FE535_Libor_USD_GBP.csv")

In [46]:
print("Libor Rates")
print("===========")
print("1M FWD",libor_rates.iloc[-1]["US0001M Index"] - libor_rates.iloc[-1]["BP0001M Index"])
print("3M FWD",libor_rates.iloc[-1]["US0003M Index"] - libor_rates.iloc[-1]["BP0003M Index"])
print("6M FWD",libor_rates.iloc[-1]["US0006M Index"] - libor_rates.iloc[-1]["BP0006M Index"])
print()
print("Calibrated Theta")
print("================")
print("1M FWD",forward_quotes["Theta"][0])
print("3M FWD",forward_quotes["Theta"][2])
print("6M FWD",forward_quotes["Theta"][5])

Libor Rates
1M FWD 1.23548
3M FWD 0.28024999999999967
6M FWD 1.0980300000000005

Calibrated Theta
1M FWD 6.156277902347894
3M FWD 7.664986326997319
6M FWD 5.217588124282754


These differences represent the deviations between the interest rate differentials implied by the LIBOR rates and the calibrated theta values for the corresponding forward contracts.

##### c) For σ, you need to download data for the daily GBP/USD exchange rate using the “GBPUSD=X” symbol from Yahoo Finance. Your data should be daily and range between 2018-01-01 and 2022-04-03. Given the adjusted prices, you need to calibrate σ using the historical returns. Note that this calibration is backward-looking, which is in line with what you did in Project 1. (5 Points)

In [50]:
import yfinance as yf
import numpy as np

# Download historical exchange rate data
start_date = '2018-01-01'
end_date = '2022-04-03'
exchange_rate_data = yf.download('GBPUSD=X', start=start_date, end=end_date)

# Calculate daily returns
exchange_rate_data['Daily_Returns'] = exchange_rate_data['Adj Close'].pct_change()

# Compute volatility (σ)
# Using a 30-day rolling window for volatility calculation
rolling_window = 30
exchange_rate_data['Volatility'] = exchange_rate_data['Daily_Returns'].rolling(window=rolling_window).std() * np.sqrt(252)

# Select the last available volatility value as the calibrated σ
calibrated_sigma = exchange_rate_data['Volatility'].dropna().iloc[-1]

print(f"Calibrated sigma (volatility): {calibrated_sigma}")


[*********************100%%**********************]  1 of 1 completed

Calibrated sigma (volatility): 0.07705895316748293


### VaR for the Unhedged (10 Points)

##### Assume that the exporter does not hedge the exchange rate risk. In this case, the exporter exchanges the GBP on the spot market upon receiving the payment in the future. Let VT denote the profit/loss (P&L) of the exporter at delivery time. Let P&L = ST −St. What is the 99% VaR of the exporter’s
##### P&L in $? (5 Points).Recall that if VT denotes the P&L at time T, then the 1 − α VaR is V aRt(VT , α) = Et[VT ] − Qt(VT , α) (3)
##### Hint: If the pound weakens relative to the dollar, then the exporter gets paid less dollars in the future. Given that the exporter expects 1.25M pounds, the future P&L depends on the future spot rate. You need to address this question using a MC simulation.

In [51]:
# Function to simulate future spot rates using GBM
def simulate_future_spot_rate(S0, mu, sigma, T, num_simulations):
    dt = 1 / 252  # Assuming daily simulations
    num_steps = int(T / dt)
    dW = np.random.normal(0, np.sqrt(dt), size=(num_simulations, num_steps))
    W = np.cumsum(dW, axis=1)
    ST = S0 * np.exp((mu - 0.5 * sigma**2) * T + sigma * np.sqrt(T) * W[:, -1])
    return ST

# Parameters
S0 = 1.2273  # Current spot rate
mu = 0  # Drift (assuming constant)
sigma = 0.1  # Calibrated volatility
T = 1  # Time to delivery in years
num_simulations = 100000  # Number of simulations

# Simulate future spot rates
future_spot_rates = simulate_future_spot_rate(S0, mu, sigma, T, num_simulations)

# Calculate P&L (VT)
PnL = future_spot_rates - S0

# Calculate 99th percentile VaR
VaR_99 = np.percentile(PnL, 1)

print(f"99% VaR of the exporter's P&L in USD: {VaR_99}")


99% VaR of the exporter's P&L in USD: -0.26075315295406476


The calculated 99% VaR of the exporter's profit/loss (P&L) in USD is approximately -0.2608. This means that there is a 99% confidence level that the exporter's P&L will not fall below this value.

Since the VaR value is negative, it indicates that there is a 99% chance that the exporter's P&L will be greater than this amount, providing a measure of downside risk for the exporter in terms of potential losses in USD.

### Unitary Hedge (30 Points)

In [57]:
# Function to simulate future spot rates using GBM
def simulate_future_spot_rate(S0, mu, sigma, T, num_simulations):
    dt = 1 / 252  # Assuming daily simulations
    num_steps = int(T / dt)
    dW = np.random.normal(0, np.sqrt(dt), size=(num_simulations, num_steps))
    W = np.cumsum(dW, axis=1)
    ST = S0 * np.exp((mu - 0.5 * sigma**2) * T + sigma * np.sqrt(T) * W[:, -1])
    return ST

# Parameters
S0 = 1.2273  # Current spot rate
mu = 0  # Drift (assuming constant)
sigma = 0.1  # Calibrated volatility
T_dec_2024 = (datetime.datetime(2024, 12, 1) - datetime.datetime.now()).days / 252  # Time to Dec 2024
T_sep_2024 = (datetime.datetime(2024, 9, 1) - datetime.datetime.now()).days / 252  # Time to Sep 2024
num_simulations = 100000  # Number of simulations

##### Suppose you use the futures contract expiring in Dec 2024. What is the 99% VaR of the P&L with unitary hedging? (10 Points)

In [58]:
# Step 1: Simulate future spot rates
future_spot_rates_dec_2024 = simulate_future_spot_rate(S0, mu, sigma, T_dec_2024, num_simulations)
# Step 2: Calculate futures price using interest rate parity formula
futures_price_dec_2024 = S0 * np.exp((mu - 0) * T_dec_2024)
# Step 3: Calculate P&L
PnL_dec_2024 = futures_price_dec_2024 - future_spot_rates_dec_2024
# Step 4: Calculate VaR
VaR_99_dec_2024 = np.percentile(PnL_dec_2024, 1)
print(f"99% VaR of P&L with Dec 2024 Futures Contract: {VaR_99_dec_2024}")

99% VaR of P&L with Dec 2024 Futures Contract: -0.267411164133277


##### Suppose instead you use the futures contract expiring in Sep 2024 (before delivery). What is the 99% VaR of the P&L now? (10 Points)

In [59]:
future_spot_rates_sept_2024 = simulate_future_spot_rate(S0, mu, sigma, T_sep_2024, num_simulations)
futures_price_sept_2024 = S0 * np.exp((mu - 0) * T_sep_2024)
PnL_sept_2024 = futures_price_sept_2024 - future_spot_rates_sept_2024
VaR_99_sept_2024 = np.percentile(PnL_sept_2024, 1)
print(f"99% VaR of P&L with Sept 2024 Futures Contract: {VaR_99_sept_2024}")


99% VaR of P&L with Sept 2024 Futures Contract: -0.1481742047049761


##### How do justify the difference in VaR when comparing your response to Part 2, Part 3 (a), and Part (b). Elaborate in terms of basis risk

The difference in VaR between the scenarios using the Dec 2024 and Sep 2024 futures contracts can be attributed to basis risk. Basis risk arises from imperfect correlation between the hedging instrument (futures contract) and the underlying exposure (GBP payment).

**Maturity Mismatch**: The Dec 2024 futures contract matures closer to the delivery date of the GBP payment compared to the Sep 2024 contract. Therefore, it better matches the timing of the underlying exposure. This results in a lower basis risk as the futures contract's expiration is closer to the delivery date of the GBP payment. Consequently, the VaR with the Dec 2024 futures contract is higher.

**Volatility Mismatch**: Since the Sep 2024 futures contract has a shorter maturity, it may have a higher sensitivity to short-term fluctuations in the spot rate compared to the Dec 2024 contract. This mismatch in volatility between the futures contract and the underlying exposure can lead to increased basis risk and hence a lower VaR.

**Liquidity Risk**: The Dec 2024 futures contract may have higher liquidity compared to the Sep 2024 contract, resulting in tighter bid-ask spreads and lower transaction costs. This can contribute to a more effective hedge and a lower VaR.

In summary, the difference in VaR between using Dec 2024 and Sep 2024 futures contracts can be attributed to the degree of basis risk associated with each hedge. The Dec 2024 contract provides a better hedge due to its closer maturity and potentially lower volatility mismatch, resulting in a higher VaR. Conversely, the Sep 2024 contract may have higher basis risk due to its shorter maturity and higher volatility mismatch, leading to a lower VaR.R.

### Hedging using ETFs (10 Points)


##### Suppose for some reason the exporter decides to use ETFs (or ETNs) to hedge currency exposure instead of using futures or forward contracts. Your task is to screen 5 different ETFs. For each ETF, provide an economic rationale behind each to serve as a GBP/USD hedge. Justify your reasoning by reporting the hedge effectiveness of each instrument. Note this is an open question without a unique answer. However, your reasoning should make sense in terms of economic mechanisms behind the GBP/USD exchange rate movement.


When selecting ETFs (or ETNs) to hedge currency exposure, it's essential to consider their correlation with the GBP/USD exchange rate and their underlying assets. Here's a rationale for selecting five different ETFs along with their potential hedge effectiveness:

**CurrencyShares British Pound Sterling Trust (FXB):**

Rationale: FXB directly tracks the movement of the British Pound Sterling (GBP) against the USD. Holding FXB can provide a direct hedge against GBP/USD exchange rate movements.

Hedge Effectiveness: FXB's effectiveness as a hedge is high since it perfectly mirrors the GBP/USD exchange rate movements.

**iShares MSCI United Kingdom ETF (EWU):**

Rationale: EWU invests in a broad range of UK equities, including large and mid-sized companies. Since the performance of UK equities is closely tied to the economic health of the UK, which influences the GBP, EWU can serve as an indirect hedge against GBP/USD movements.

Hedge Effectiveness: EWU's effectiveness as a hedge may vary depending on factors such as global market sentiment, economic conditions in the UK, and currency fluctuations relative to other currencies in the ETF's portfolio.

**Vanguard FTSE 100 ETF (VUKG.L):**

Rationale: VUKG.L tracks the performance of the FTSE 100 Index, which consists of the 100 largest companies listed on the London Stock Exchange. Since many of these companies derive a significant portion of their revenue from international markets, including the US, movements in the GBP/USD exchange rate can impact their earnings and thus the ETF's performance.

Hedge Effectiveness: VUKG.L may provide a moderate hedge against GBP/USD movements, but its effectiveness could be influenced by factors such as currency hedging strategies employed by the companies in the index and global economic conditions.

**Invesco CurrencyShares Euro Trust (FXE):**

Rationale: While FXE directly tracks the Euro (EUR) against the USD, it can indirectly provide exposure to the GBP/USD exchange rate since the GBP and EUR often move in tandem due to their geographical proximity and economic ties.

Hedge Effectiveness: FXE's effectiveness as a hedge against GBP/USD movements may be moderate, as it depends on the correlation between EUR/USD and GBP/USD exchange rates.

**SPDR Gold Shares (GLD):**

Rationale: GLD tracks the price of gold, which is often viewed as a safe-haven asset. During periods of economic uncertainty or currency devaluation, investors may flock to gold, causing its price to rise. As a result, holding GLD can provide a hedge against currency depreciation, including movements in GBP/USD.

Hedge Effectiveness: GLD's effectiveness as a hedge against GBP/USD movements may vary, with its performance influenced by factors such as geopolitical tensions, inflation expectations, and US dollar strength.